In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec

In [33]:
data = pd.read_csv('/content/drive/MyDrive/data.csv')

In [34]:
rows_to_drop = data[data['Label'] == 'entertainment'].index
data = data.drop(rows_to_drop)

In [35]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [36]:
X = train_data['Headline'].values
y = train_data['Label'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
max_length = max([len(seq) for seq in sequences])
X_pad = pad_sequences(sequences, maxlen=max_length)

In [37]:
X = val_data['Headline'].values
y_val = val_data['Label'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index_val = tokenizer.word_index
max_length_val = max([len(seq) for seq in sequences])
X_val = pad_sequences(sequences, maxlen=max_length)

In [38]:
from gensim.models import Word2Vec, KeyedVectors

In [39]:
word2vec_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True)
embedding_matrix = np.zeros((len(word_index) + 1, word2vec_model.vector_size))
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

In [40]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y_val = label_encoder.fit_transform(y_val)

In [15]:
#
# model = Sequential()
# model.add(Embedding(input_dim=len(word_index) + 1, output_dim=word2vec_model.vector_size, weights=[embedding_matrix], input_length=max_length, trainable=False))
# model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(units=len(set(y)), activation='softmax'))

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.fit(X_pad, y, epochs=10, batch_size=64, validation_data=(X_val, y_val))

ValueError: Layer embedding_3 weight shape (956, 300) is not compatible with provided weight shape (2788, 300).

In [41]:
from keras.layers import Dropout

# RNN with Dropout
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=word2vec_model.vector_size, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(units=64, recurrent_dropout=0.2))
model.add(Dropout(0.8))
model.add(Dense(units=len(set(y)), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_pad, y, epochs=5 , batch_size=64, validation_data=(X_val, y_val))

Epoch 1/5
21/21 [==============================] - 5s 59ms/step - loss: 1.3307 - accuracy: 0.3706 - val_loss: 1.3925 - val_accuracy: 0.2971
Epoch 2/5
21/21 [==============================] - 1s 35ms/step - loss: 1.1692 - accuracy: 0.5905 - val_loss: 1.4254 - val_accuracy: 0.2935
Epoch 3/5
21/21 [==============================] - 1s 34ms/step - loss: 0.9250 - accuracy: 0.6908 - val_loss: 1.6379 - val_accuracy: 0.2971
Epoch 4/5
21/21 [==============================] - 1s 36ms/step - loss: 0.6865 - accuracy: 0.7863 - val_loss: 2.0638 - val_accuracy: 0.2790
Epoch 5/5
21/21 [==============================] - 1s 32ms/step - loss: 0.5307 - accuracy: 0.8291 - val_loss: 2.4339 - val_accuracy: 0.2717


In [42]:
#test
X = test_data['Headline'].values
y_test = test_data['Label'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index_test = tokenizer.word_index
max_length_test = max([len(seq) for seq in sequences])
X_test = pad_sequences(sequences, maxlen=max_length)

In [43]:
y_test = label_encoder.fit_transform(y_test)
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

9/9 [==============================] - 0s 8ms/step - loss: 2.5047 - accuracy: 0.2246
Test Loss: 2.5047340393066406
Test Accuracy: 0.2246376872062683
